In [41]:
import requests
import pandas as pd
from datetime import timedelta
pd.set_option('display.max_columns', 100) 
API_KEY = "qs6/qOme90jX/egZWGD1kNe2KIZnhHulZAFcBnvvQH1F65nDEVOSxAEvv/LsSy09"

In [34]:
def get_fbs_scores(season):
    url = "https://api.collegefootballdata.com/games"
    params = {
        "year": season,
        "seasonType": "both",
        "division": "fbs"
    }
    headers = {
        "Authorization": f"Bearer {API_KEY}"
    }

    response = requests.get(url, params=params, headers=headers)
    response.raise_for_status()
    games = response.json()

    df = pd.DataFrame(games)
    # Only the selected columns
    cols = [
        "id",
        "week",
        "seasonType",
        "startDate",
        "completed",
        "venue",
        "neutralSite",
        "conferenceGame",
        "homeTeam",
        "homeClassification",
        "homeConference",
        "homePoints",
        "homeLineScores",
        "awayTeam",
        "awayClassification",
        "awayConference",
        "awayPoints",
        "awayLineScores",
        "excitementIndex"
    ]
    df_out = df[cols]
    return df_out

# Example usage: 2024, Week 1
df_scores = get_fbs_scores(2024)
df_scores  # In Jupyter, this will show the nicely formatted table!


,id,week,seasonType,startDate,completed,venue,neutralSite,conferenceGame,homeTeam,homeClassification,homeConference,homePoints,homeLineScores,awayTeam,awayClassification,awayConference,awayPoints,awayLineScores,excitementIndex
0,401693677,1,regular,2024-08-24T04:00:00.000Z,True,None,True,False,Lincoln (CA),ii,Independent DII,7.0,"[0, 0, 0, 7]",College of Idaho,None,None,45.0,"[14, 7, 14, 10]",NaN
1,401635525,1,regular,2024-08-24T16:00:00.000Z,True,Aviva Stadium,True,True,Georgia Tech,fbs,ACC,24.0,"[7, 7, 0, 10]",Florida State,fbs,ACC,21.0,"[8, 6, 0, 7]",7.822224
2,401654665,1,regular,2024-08-24T19:30:00.000Z,True,"Memorial Stadium (Stephenville, TX)",False,False,Tarleton State,fcs,UAC,26.0,"[16, 10, 0, 0]",McNeese,fcs,Southland,23.0,"[7, 0, 3, 13]",5.564025
3,401643697,1,regular,2024-08-24T20:00:00.000Z,True,University Stadium (NM),False,False,New Mexico,fbs,Mountain West,31.0,"[10, 14, 7, 0]",Montana State,fcs,Big Sky,35.0,"[0, 14, 0, 21]",4.927318
4,401654662,1,regular,2024-08-24T23:00:00.000Z,True,Cramton Bowl,True,False,Southeast Missouri State,fcs,Big South-OVC,37.0,"[7, 6, 8, 16]",North Alabama,fcs,UAC,15.0,"[8, 7, 0, 0]",7.171901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3796,401741208,1,postseason,2025-01-06T01:00:00.000Z,True,BBVA Compass Stadium,True,False,North Central College,iii,CCIW,41.0,"[7, 7, 7, 20]",University of Mount Union,iii,Ohio,25.0,"[7, 3, 7, 8]",NaN
3797,401680057,1,postseason,2025-01-07T00:00:00.000Z,True,Toyota Stadium,True,False,Montana State,fcs,Big Sky,32.0,"[0, 3, 15, 14]",North Dakota State,fcs,MVFC,35.0,"[14, 7, 0, 14]",4.820656
3798,401677189,1,postseason,2025-01-10T00:30:00.000Z,True,Hard Rock Stadium,True,False,Penn State,fbs,Big Ten,24.0,"[0, 10, 0, 14]",Notre Dame,fbs,FBS Independents,27.0,"[0, 3, 7, 17]",8.450324
3799,401677191,1,postseason,2025-01-11T00:30:00.000Z,True,AT&T Stadium,True,False,Texas,fbs,SEC,14.0,"[0, 7, 7, 0]",Ohio State,fbs,Big Ten,28.0,"[7, 7, 0, 14]",6.155203


In [55]:
def clean_scores_df(df):
    df = df.copy()
    
    # Rename columns (including Venue)
    df = df.rename(columns={
        'id': 'Game_ID',
        'week': 'Week',
        'seasonType': 'Season_Type',
        'startDate': 'Start_Date',
        'venue': 'Venue',
        'completed': 'Completed',
        'neutralSite': 'Neutral_Site',
        'conferenceGame': 'Conference_Game',
        'homeTeam': 'Home_Team',
        'homeClassification': 'Home_Classification',
        'homeConference': 'Home_Conference',
        'homePoints': 'Home_Pts',
        'homeLineScores': 'Home_Line_Scores',
        'awayTeam': 'Away_Team',
        'awayClassification': 'Away_Classification',
        'awayConference': 'Away_Conference',
        'awayPoints': 'Away_Pts',
        'awayLineScores': 'Away_Line_Scores',
        'excitementIndex': 'Excitement'
    })
    
    # Capitalize all values in Season_Type
    df['Season_Type'] = df['Season_Type'].str.title()
    
    # Convert Start_Date to datetime (UTC)
    df['Start_DateTime'] = pd.to_datetime(df['Start_Date'], utc=True)
    df['Start_DateTime_EST'] = df['Start_DateTime'].dt.tz_convert('US/Eastern')

    # Create separate Start_Date and Start_Time columns in Eastern
    # Format date as "Fri, Aug. 29, 2024"
    df['Start_Date'] = df['Start_DateTime_EST'].dt.strftime('%a, %b. %d, %Y')
    df['Start_Time'] = df['Start_DateTime_EST'].dt.strftime('%H:%M')

    # Completed and Neutral_Site to Yes/No
    for col in ['Completed', 'Neutral_Site']:
        df[col] = df[col].apply(lambda x: 'Yes' if x else 'No')
    
    # Conference_Game: Non-Conf if False, else use conference name (from home or away)
    def get_conf(row):
        if not row['Conference_Game']:
            return 'Non-Conf'
        return row['Home_Conference'] if row['Home_Conference'] else row['Away_Conference']
    df['Conference_Game'] = df.apply(get_conf, axis=1)
    
    # Home/Away_Classification fully uppercase
    df['Home_Classification'] = df['Home_Classification'].str.upper()
    df['Away_Classification'] = df['Away_Classification'].str.upper()
    
    # Filter: at least one FBS team
    df = df[(df['Home_Classification'] == 'FBS') | (df['Away_Classification'] == 'FBS')]
    
    # Points as integer (remove decimals)
    df['Home_Pts'] = pd.to_numeric(df['Home_Pts'], errors='coerce').fillna(0).astype(int)
    df['Away_Pts'] = pd.to_numeric(df['Away_Pts'], errors='coerce').fillna(0).astype(int)
    
    # Excitement_Index: one decimal
    df['Excitement'] = pd.to_numeric(df['Excitement'], errors='coerce').round(1)
    
    # Keep only last 6 digits of Game_ID
    df['Game_ID'] = df['Game_ID'].astype(str).str[-6:]
    
    # Break out line scores (handle missing or short lists gracefully)
    def get_q(scores, idx):
        try:
            return int(scores[idx])
        except Exception:
            return 0

    # Home line scores
    df['Home_1Q'] = df['Home_Line_Scores'].apply(lambda x: get_q(x, 0) if isinstance(x, list) else 0)
    df['Home_2Q'] = df['Home_Line_Scores'].apply(lambda x: get_q(x, 1) if isinstance(x, list) else 0)
    df['Home_3Q'] = df['Home_Line_Scores'].apply(lambda x: get_q(x, 2) if isinstance(x, list) else 0)
    df['Home_4Q'] = df['Home_Line_Scores'].apply(lambda x: get_q(x, 3) if isinstance(x, list) else 0)

    # Away line scores
    df['Away_1Q'] = df['Away_Line_Scores'].apply(lambda x: get_q(x, 0) if isinstance(x, list) else 0)
    df['Away_2Q'] = df['Away_Line_Scores'].apply(lambda x: get_q(x, 1) if isinstance(x, list) else 0)
    df['Away_3Q'] = df['Away_Line_Scores'].apply(lambda x: get_q(x, 2) if isinstance(x, list) else 0)
    df['Away_4Q'] = df['Away_Line_Scores'].apply(lambda x: get_q(x, 3) if isinstance(x, list) else 0)

    mask = df['Season_Type'] == 'Regular'
    df_reg = df[mask].sort_values('Start_DateTime_EST')
    week = 0
    week_cuts = []

    # Find the first week start (first game's date, but with time set to 00:00:00, next Tuesday)
    first_date = df_reg.iloc[0]['Start_DateTime_EST']
    # Move to Tuesday midnight of that week
    first_tuesday = first_date + timedelta(days=(1 - first_date.weekday()) % 7)
    first_tuesday = first_tuesday.replace(hour=0, minute=0, second=0, microsecond=0)
    if first_date > first_tuesday:
        current_week_cut = first_tuesday
    else:
        # If first game is before Tuesday, that's week 0
        current_week_cut = first_tuesday

    week_list = []
    for dt in df_reg['Start_DateTime_EST']:
        if dt >= current_week_cut:
            week += 1
            current_week_cut += timedelta(days=7)
        week_list.append(week)
    df.loc[mask, 'Week'] = week_list

    # For all non-regular season, set week as 'Post-Season'
    df['Week'] = df['Week'].astype(str)
    df.loc[df['Season_Type'] != 'Regular', 'Week'] = 'Post-Season'
    df['Week'] = df['Week'].astype(str)

    # Reorder columns (optional)
    final_cols = [
        'Week', 'Start_Date', 'Start_Time', 'Venue',
        'Completed', 'Neutral_Site', 'Conference_Game', 'Home_Team', 'Home_Pts',
        'Home_1Q', 'Home_2Q', 'Home_3Q', 'Home_4Q', 'Away_Team', 'Away_Pts',
        'Away_1Q', 'Away_2Q', 'Away_3Q', 'Away_4Q', 'Excitement'
    ]
    df = df[final_cols]

    return df


In [56]:
df_clean = clean_scores_df(df_scores)
df_clean.head()

,Week,Start_Date,Start_Time,Venue,Completed,Neutral_Site,Conference_Game,Home_Team,Home_Pts,Home_1Q,Home_2Q,Home_3Q,Home_4Q,Away_Team,Away_Pts,Away_1Q,Away_2Q,Away_3Q,Away_4Q,Excitement
1,0,"Sat, Aug. 24, 2024",12:00,Aviva Stadium,Yes,Yes,ACC,Georgia Tech,24,7,7,0,10,Florida State,21,8,6,0,7,7.8
3,0,"Sat, Aug. 24, 2024",16:00,University Stadium (NM),Yes,No,Non-Conf,New Mexico,31,10,14,7,0,Montana State,35,0,14,0,21,4.9
6,0,"Sat, Aug. 24, 2024",20:00,Mackay Stadium,Yes,No,Non-Conf,Nevada,24,7,10,7,0,SMU,29,0,10,3,16,7.7
7,0,"Sat, Aug. 24, 2024",23:59,Clarence T.C. Ching Athletics Complex,Yes,No,Non-Conf,Hawai'i,35,14,0,7,14,Delaware State,14,0,7,7,0,5.2
11,1,"Thu, Aug. 29, 2024",18:00,SHI Stadium,Yes,No,Non-Conf,Rutgers,44,7,10,14,13,Howard,7,0,7,0,0,4.7


In [59]:
df_scores_2025 = get_fbs_scores(2025)
df_scores_2025

,id,week,seasonType,startDate,completed,venue,neutralSite,conferenceGame,homeTeam,homeClassification,homeConference,homePoints,homeLineScores,awayTeam,awayClassification,awayConference,awayPoints,awayLineScores,excitementIndex
0,401767126,1,regular,2025-08-23T04:00:00.000Z,False,Hillsboro Stadium,False,False,Portland State,fcs,Big Sky,None,None,Tarleton State,fcs,UAC,None,None,None
1,401767476,1,regular,2025-08-23T04:00:00.000Z,False,Manning Field at John L. Guidry Stadium,False,True,Nicholls,fcs,Southland,None,None,Incarnate Word,fcs,Southland,None,None,None
2,401756846,1,regular,2025-08-23T16:00:00.000Z,False,Aviva Stadium,True,True,Iowa State,fbs,Big 12,None,None,Kansas State,fbs,Big 12,None,None,None
3,401756847,1,regular,2025-08-23T22:30:00.000Z,False,Memorial Stadium,False,False,Kansas,fbs,Big 12,None,None,Fresno State,fbs,Mountain West,None,None,None
4,401767135,1,regular,2025-08-23T23:00:00.000Z,False,Cramton Bowl,True,False,Mercer,fcs,Southern,None,None,UC Davis,fcs,Big Sky,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1593,401752921,14,regular,2025-11-29T17:00:00.000Z,False,Michigan Stadium,False,True,Michigan,fbs,Big Ten,None,None,Ohio State,fbs,Big Ten,None,None,None
1594,401757313,14,regular,2025-11-29T20:30:00.000Z,False,Williams Stadium (VA),False,True,Liberty,fbs,Conference USA,None,None,Kennesaw State,fbs,Conference USA,None,None,None
1595,401752946,14,regular,2025-11-29T23:30:00.000Z,False,Gesa Field,False,True,Washington State,fbs,Pac-12,None,None,Oregon State,fbs,Pac-12,None,None,None
1596,401760425,14,regular,2025-11-30T02:00:00.000Z,False,Mackay Stadium,False,True,Nevada,fbs,Mountain West,None,None,UNLV,fbs,Mountain West,None,None,None


In [61]:
df_clean_2025 = clean_scores_df(df_scores_2025)
df_clean_2025.head()

,Week,Start_Date,Start_Time,Venue,Completed,Neutral_Site,Conference_Game,Home_Team,Home_Pts,Home_1Q,Home_2Q,Home_3Q,Home_4Q,Away_Team,Away_Pts,Away_1Q,Away_2Q,Away_3Q,Away_4Q,Excitement
2,0,"Sat, Aug. 23, 2025",12:00,Aviva Stadium,No,Yes,Big 12,Iowa State,0,0,0,0,0,Kansas State,0,0,0,0,0,NaN
3,0,"Sat, Aug. 23, 2025",18:30,Memorial Stadium,No,No,Non-Conf,Kansas,0,0,0,0,0,Fresno State,0,0,0,0,0,NaN
5,0,"Sat, Aug. 23, 2025",19:00,Houchens Industries-L.T. Smith Stadium,No,No,Conference USA,Western Kentucky,0,0,0,0,0,Sam Houston,0,0,0,0,0,NaN
6,0,"Sat, Aug. 23, 2025",19:30,Clarence T.C. Ching Athletics Complex,No,No,Non-Conf,Hawai'i,0,0,0,0,0,Stanford,0,0,0,0,0,NaN
11,1,"Thu, Aug. 28, 2025",00:00,Snapdragon Stadium,No,No,Non-Conf,San Diego State,0,0,0,0,0,Stony Brook,0,0,0,0,0,NaN


In [82]:
def make_game_card(row):
    # Determine winner/loser and their score colors (only for outside scores)
    if row['Home_Pts'] > row['Away_Pts']:
        home_score_color = "#155724"  # dark green
        away_score_color = "#A93226"  # dark red
    elif row['Home_Pts'] < row['Away_Pts']:
        home_score_color = "#A93226"
        away_score_color = "#155724"
    else:
        home_score_color = away_score_color = "#555"  # Tie (gray)
    
    card = f"""
    <div style="border:1px solid #ddd; border-radius:10px; padding:16px; margin-bottom:20px; background:#FFFBF2; max-width:640px;">
        <div style="display:flex; justify-content:space-between; align-items:center; font-family:sans-serif;">
            <div style="text-align:center; flex:1;">
                <span style="font-weight:bold; font-size:22px;">{row['Away_Team']}</span><br>
                <span style="font-size:32px; color:{away_score_color}; font-weight:bold;">{row['Away_Pts']}</span>
            </div>
            <div style="text-align:center; flex:2;">
                <table style="margin:auto; font-size:14px; border-collapse:separate; border-spacing:0; border: 1px solid #000; border-radius:8px; overflow:hidden;">
                  <tr style='background-color:#eaf6ff;'>
                    <th style="font-weight:bold; padding:4px 8px;"></th>
                    <th style="font-weight:bold; padding:4px 8px;">1Q</th>
                    <th style="font-weight:bold; padding:4px 8px;">2Q</th>
                    <th style="font-weight:bold; padding:4px 8px;">3Q</th>
                    <th style="font-weight:bold; padding:4px 8px;">4Q</th>
                    <th style="font-weight:bold; padding:4px 8px;">T</th>
                  </tr>
                  <tr style="background-color:#fff;">
                    <td style='text-align:right; padding:4px 8px;'>{row['Away_Team']}</td>
                    <td style='text-align:center; color:#000;'>{row['Away_1Q']}</td>
                    <td style='text-align:center; color:#000;'>{row['Away_2Q']}</td>
                    <td style='text-align:center; color:#000;'>{row['Away_3Q']}</td>
                    <td style='text-align:center; color:#000;'>{row['Away_4Q']}</td>
                    <td style='font-weight:bold; text-align:center; color:#000;'>{row['Away_Pts']}</td>
                  </tr>
                  <tr style="background-color:#eaf6ff;">
                    <td style='text-align:right; padding:4px 8px;'>{row['Home_Team']}</td>
                    <td style='text-align:center; color:#000;'>{row['Home_1Q']}</td>
                    <td style='text-align:center; color:#000;'>{row['Home_2Q']}</td>
                    <td style='text-align:center; color:#000;'>{row['Home_3Q']}</td>
                    <td style='text-align:center; color:#000;'>{row['Home_4Q']}</td>
                    <td style='font-weight:bold; text-align:center; color:#000;'>{row['Home_Pts']}</td>
                  </tr>
                </table>
                <div style="margin-top:10px; color:#333; font-size:13px;">
                    {row['Start_Date']} &bull; {row['Start_Time']} ET<br>
                    Venue: <b>{row['Venue']}</b><br>
                    Conference: <b>{row['Conference_Game']}</b> &nbsp;|&nbsp; Excitement: <b>{row['Excitement']}</b>
                </div>
            </div>
            <div style="text-align:center; flex:1;">
                <span style="font-weight:bold; font-size:22px;">{row['Home_Team']}</span><br>
                <span style="font-size:32px; color:{home_score_color}; font-weight:bold;">{row['Home_Pts']}</span>
            </div>
        </div>
    </div>
    """
    return card


In [83]:
# Filter for completed games on Aug 24, 2024
df_games = df_clean[
    (df_clean['Completed'] == 'Yes') &
    (df_clean['Start_Date'] == 'Sat, Aug. 24, 2024')
]


In [84]:
report_html = ""
for _, row in df_games.iterrows():
    report_html += make_game_card(row)
from IPython.display import display, HTML
display(HTML(report_html))


,1Q,2Q,3Q,4Q,T
Florida State,8,6,0,7,21
Georgia Tech,7,7,0,10,24
,1Q,2Q,3Q,4Q,T
Montana State,0,14,0,21,35
New Mexico,10,14,7,0,31
,1Q,2Q,3Q,4Q,T
SMU,0,10,3,16,29
Nevada,7,10,7,0,24
,1Q,2Q,3Q,4Q,T
Delaware State,0,7,7,0,14


In [85]:
df_clean['Home_Team'].unique()

array(['Georgia Tech', 'New Mexico', 'Nevada', "Hawai'i", 'Rutgers',
       'Wake Forest', 'UCF', 'NC State', 'UL Monroe', 'Buffalo',
       'Bowling Green', 'Central Michigan', 'Arkansas', 'Toledo',
       'Missouri', 'Minnesota', 'UAB', 'Tulane', 'Tulsa', 'Kansas',
       'Colorado', 'Jacksonville State', 'Utah', 'Illinois',
       'San José State', 'Army', 'Oklahoma', 'Michigan State', 'Duke',
       'Wisconsin', 'Stanford', 'Utah State', 'Iowa', 'Purdue', 'Navy',
       'Louisville', 'Vanderbilt', 'West Virginia', 'Maryland',
       'Pittsburgh', 'Georgia', 'Tennessee', 'Oklahoma State',
       'Cincinnati', 'Washington State', 'Nebraska', 'Iowa State',
       'App State', 'Northern Illinois', 'Air Force', 'Massachusetts',
       'Indiana', 'Ohio State', 'Florida', 'Syracuse', 'Northwestern',
       'UTSA', 'Texas', 'Georgia Southern', 'South Carolina',
       'California', 'Marshall', 'South Alabama', 'East Carolina',
       'Mississippi State', 'Liberty', 'Virginia', 'Oregon Stat

In [86]:
import os

logo_dir = 'FBS-Logo-Library-main/CFB Logos'
all_logo_files = [f for f in os.listdir(logo_dir) if f.endswith('.png')]

# Try to match based on team names
auto_team_to_logo = {}
for team in df_clean['Home_Team'].unique():
    # Replace spaces/hyphens with underscores, remove special chars
    possible_files = [f for f in all_logo_files if team.replace(" ", "_").replace("-", "_") in f]
    if possible_files:
        auto_team_to_logo[team] = possible_files[0]
    else:
        print(f"No match for {team}")


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'FBS-Logo-Library-main/CFB Logos'